# 성능튜닝

## 1.환경준비

### (1) import

In [1]:
#라이브러리들을 불러오자.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 전처리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 모델링
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import * 

import warnings    # 경고메시지 제외
warnings.filterwarnings(action='ignore')

### (2) 데이터 준비

* 변수설명
    * COLLEGE : 대학 졸업여부
    * INCOME : 연수입
    * OVERAGE : 월평균 초과사용 시간(분)
    * LEFTOVER : 월평균 잔여시간비율(%)
    * HOUSE : 집값
    * HANDSET_PRICE : 스마트폰 가격
    * OVER_15MINS_CALLS_PER_MONTH : 월평균 장기통화(15분이상) 횟수
    * AVERAGE_CALL_DURATION : 평균 통화 시간
    * REPORTED_SATISFACTION : 만족도 설문조사 결과
    * REPORTED_USAGE_LEVEL : 사용도 자가진단 결과
    * CONSIDERING_CHANGE_OF_PLAN : 향후 변경계획 설문조사 결과
    * CHURN : 이탈(번호이동) 여부 (1-이탈, 0-잔류, Target 변수)


In [2]:
# 데이터를 불러옵시다.
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/mobile_cust_churn.csv'
data = pd.read_csv(path)
data = data.sample(5000, random_state = 2022)
data['CHURN'] = data['CHURN'].map({'LEAVE':1, 'STAY':0}) # 1, 0으로 변경 (sklearn에서는 가변수화할 필요 없음),  statsmodels/tensorflow(keras)에서는 반드시 필요
data.head()

,id,COLLEGE,INCOME,OVERAGE,LEFTOVER,HOUSE,HANDSET_PRICE,OVER_15MINS_CALLS_PER_MONTH,AVERAGE_CALL_DURATION,REPORTED_SATISFACTION,REPORTED_USAGE_LEVEL,CONSIDERING_CHANGE_OF_PLAN,CHURN
3178,3179,0,119512,51,31,248566,229,5,2,very_sat,very_high,considering,1
14926,14927,1,142144,192,15,774317,581,29,4,unsat,very_little,never_thought,1
15116,15117,1,142308,0,79,306426,497,1,1,sat,little,considering,0
12733,12734,1,113385,0,0,333599,819,1,6,very_unsat,very_high,considering,1
14032,14033,1,90348,209,10,637286,360,26,4,unsat,little,actively_looking_into_it,0


## 2.데이터 준비

### (1) 데이터 정리

In [3]:
drop_cols = ['id']
data.drop(drop_cols, axis = 1, inplace = True )

### (2) 데이터분할1 : x, y 나누기

In [4]:
target = 'CHURN'
x = data.drop(target, axis = 1)
y = data.loc[:, target]

### (3) NA 조치

### (4) 가변수화

In [5]:
dumm_cols = ['REPORTED_SATISFACTION','REPORTED_USAGE_LEVEL','CONSIDERING_CHANGE_OF_PLAN']
x = pd.get_dummies(x, columns = dumm_cols, drop_first = True)

### (5) 데이터분할2 : train : validation 나누기

In [6]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = .3, random_state = 20)

### (6) Scaling

In [7]:
scaler = MinMaxScaler()
x_train_s = scaler.fit_transform(x_train)
x_val_s = scaler.transform(x_val)

## 3.선형모델 튜닝

Logistic Regression : 전진선택법
* 변수를 하나씩 늘려가면서
* AIC를 가장 낮추는 모델 찾기

### (1) 전진선택을 수행할 함수 만들기( **로지스틱 회귀** 용)

In [8]:
# 아래 함수는 로지스틱 회귀를 위한 전진선택법 함수 입니다.
import statsmodels.api as sm

def forward_stepwise_logistic(x_train, y_train):

    # 변수목록, 선택된 변수 목록, 단계별 모델과 AIC 저장소 정의
    features = list(x_train)
    selected = []
    step_df = pd.DataFrame({ 'step':[], 'feature':[],'aic':[]})

    # 
    for s in range(0, len(features)) :
        result =  { 'step':[], 'feature':[],'aic':[]}

        # 변수 목록에서 변수 한개씩 뽑아서 모델에 추가
        for f in features :
            vars = selected + [f]
            x_tr = x_train[vars]
            model = sm.Logit(y_train, x_tr).fit()
            result['step'].append(s+1)
            result['feature'].append(vars)
            result['aic'].append(model.aic)
        
        # 모델별 aic 집계
        temp = pd.DataFrame(result).sort_values('aic').reset_index(drop = True)

        # 만약 이전 aic보다 새로운 aic 가 크다면 멈추기
        if step_df['aic'].min() < temp['aic'].min() :
            break
        step_df = pd.concat([step_df, temp], axis = 0).reset_index(drop = True)

        # 선택된 변수 제거
        v = temp.loc[0,'feature'][s]
        features.remove(v)

        selected.append(v)
    
    # 선택된 변수와 step_df 결과 반환
    return selected, step_df

### (2) 전진선택법 수행

In [9]:
vars, result = forward_stepwise_logistic(x_train, y_train)

Optimization terminated successfully.
         Current function value: 0.693075
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693062
         Iterations 2
Optimization terminated successfully.
         Current function value: 0.683528
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693007
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.684909
         Iterations 2
Optimization terminated successfully.
         Current function value: 0.693042
         Iterations 2
Optimization terminated successfully.
         Current function value: 0.686899
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692565
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692806
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693137
  

* 선택된 변수

In [10]:
vars

['OVERAGE',
 'HOUSE',
 'HANDSET_PRICE',
 'LEFTOVER',
 'REPORTED_SATISFACTION_very_sat',
 'INCOME',
 'REPORTED_SATISFACTION_sat']

In [11]:
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

### (3) 모델링

* 전체 변수 

In [12]:
m1 = LogisticRegression()
m1.fit(x_train, y_train)
p1 = m1.predict(x_val)

print(accuracy_score(y_val, p1))
print(classification_report(y_val, p1))

0.6333333333333333
              precision    recall  f1-score   support

           0       0.62      0.68      0.65       738
           1       0.65      0.59      0.62       762

    accuracy                           0.63      1500
   macro avg       0.63      0.63      0.63      1500
weighted avg       0.64      0.63      0.63      1500



* 전진선택법 변수

In [13]:
m2 = LogisticRegression()
m2.fit(x_train[vars], y_train)
p2 = m2.predict(x_val[vars])

print(accuracy_score(y_val, p2))
print(classification_report(y_val, p2))

0.634
              precision    recall  f1-score   support

           0       0.62      0.68      0.65       738
           1       0.66      0.59      0.62       762

    accuracy                           0.63      1500
   macro avg       0.64      0.63      0.63      1500
weighted avg       0.64      0.63      0.63      1500



## 4.하이퍼파라미터 튜닝

### (1) 필요한 함수 불러오기 

In [14]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

### (2) Random Search

① 값의 범위를 지정한다.  
② 모델 선언(시도 횟수 지정)  
③ 모델링(값의 범위 내에서 시도 횟수만큼 랜덤하게 선택해서 시도한다.)  
④ 가장 성능이 좋은 값을 선정


#### ① 값의 범위를 지정한다.

In [15]:
# dictionary형태로 선언
params = { 'n_neighbors' : range(1,51), 'metric' : ['euclidean', 'manhattan']  }
params

{'n_neighbors': range(1, 51), 'metric': ['euclidean', 'manhattan']}

#### ② 모델 선언

In [16]:
# 기본모델
model = KNeighborsClassifier()

# Random Search 설정.
model_rs = RandomizedSearchCV(model
                            , params              # hyperparameter 범위 지정.
                            , cv=5                    # k-fold Cross Validation
                            , n_iter=5                # Random하게 시도할 횟수
                            )

#### ③ 모델링

In [17]:
# 학습 : model이 아니라 model_rs
model_rs.fit(x_train_s, y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(), n_iter=5,
                   param_distributions={'metric': ['euclidean', 'manhattan'],
                                        'n_neighbors': range(1, 51)})

In [18]:
# 튜닝 결과
model_rs.cv_results_

{'mean_fit_time': array([0.00242186, 0.00235357, 0.00233359, 0.00215487, 0.00164871]),
 'std_fit_time': array([0.00132386, 0.0008226 , 0.00032343, 0.00071886, 0.00044862]),
 'mean_score_time': array([0.17676244, 0.117489  , 0.14517279, 0.12142558, 0.14835806]),
 'std_score_time': array([0.06125358, 0.01478591, 0.01064934, 0.00560152, 0.01079946]),
 'param_n_neighbors': masked_array(data=[26, 45, 37, 7, 45],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_metric': masked_array(data=['manhattan', 'euclidean', 'manhattan', 'euclidean',
                    'manhattan'],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_neighbors': 26, 'metric': 'manhattan'},
  {'n_neighbors': 45, 'metric': 'euclidean'},
  {'n_neighbors': 37, 'metric': 'manhattan'},
  {'n_neighbors': 7, 'metric': 'euclidean'},
  {'n_neighbors': 45, 'metric': 'manhattan'}],
 'split0_te

In [19]:
model_rs.cv_results_['params']

[{'n_neighbors': 26, 'metric': 'manhattan'},
 {'n_neighbors': 45, 'metric': 'euclidean'},
 {'n_neighbors': 37, 'metric': 'manhattan'},
 {'n_neighbors': 7, 'metric': 'euclidean'},
 {'n_neighbors': 45, 'metric': 'manhattan'}]

In [20]:
model_rs.cv_results_['mean_test_score']

array([0.61771429, 0.59      , 0.62771429, 0.57714286, 0.62142857])

In [21]:
# 최적의 파라미터
model_rs.best_params_

{'n_neighbors': 37, 'metric': 'manhattan'}

In [22]:
# 그때의 성능
model_rs.best_score_

0.6277142857142857

In [23]:
# best 모델로 예측 및 평가
pred = model_rs.predict(x_val_s)
print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       0.59      0.74      0.66       738
           1       0.66      0.51      0.58       762

    accuracy                           0.62      1500
   macro avg       0.63      0.62      0.62      1500
weighted avg       0.63      0.62      0.62      1500



### (3) 실습 : Random Search

* decision tree로 튜닝을 시도해 봅시다.
    * max_depth : 1~10
    * min_samples_leaf : 10 ~ 100

#### ① 값의 범위를 지정한다.

In [24]:
# dictionary형태로 선언
params = { 'max_depth' : range(1, 11), 'min_samples_leaf' : range(10, 101, 10) }
params

{'max_depth': range(1, 11), 'min_samples_leaf': range(10, 101, 10)}

#### ② 모델 선언

In [25]:
model_dt = DecisionTreeClassifier()

random_dt = RandomizedSearchCV( model_dt,
                                params,
                                cv=5,
                                n_iter=10
                              )

#### ③ 모델링

In [26]:
random_dt.fit(x_train, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(),
                   param_distributions={'max_depth': range(1, 11),
                                        'min_samples_leaf': range(10, 101, 10)})

In [27]:
random_dt.cv_results_

{'mean_fit_time': array([0.0229444 , 0.00753055, 0.01309013, 0.01646419, 0.01029286,
        0.01169648, 0.02117491, 0.00625792, 0.01326976, 0.01587529]),
 'std_fit_time': array([0.00389714, 0.00137963, 0.00242957, 0.00158586, 0.0003259 ,
        0.00037124, 0.00090423, 0.00100052, 0.00086239, 0.00081711]),
 'mean_score_time': array([0.00771313, 0.00361533, 0.00413394, 0.0037396 , 0.00319071,
        0.00303946, 0.00313182, 0.00331178, 0.00269065, 0.00294113]),
 'std_score_time': array([0.0040283 , 0.00078789, 0.00119596, 0.00105262, 0.00039995,
        0.00064032, 0.0002476 , 0.00096198, 0.00039234, 0.00058094]),
 'param_min_samples_leaf': masked_array(data=[20, 60, 50, 50, 80, 90, 10, 10, 10, 50],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[6, 1, 3, 5, 3, 4, 10, 1, 4, 8],
              mask=[False, False, False, False, False, Fa

In [28]:
random_dt.best_params_

{'min_samples_leaf': 90, 'max_depth': 4}

In [29]:
random_dt.best_score_

0.6974285714285715

In [30]:
pred_dt = random_dt.predict(x_val)
print(classification_report(y_val, pred_dt))

              precision    recall  f1-score   support

           0       0.74      0.61      0.67       738
           1       0.68      0.79      0.73       762

    accuracy                           0.70      1500
   macro avg       0.71      0.70      0.70      1500
weighted avg       0.71      0.70      0.70      1500



### (4) Grid Search

① 값의 범위를 지정한다.  
② 모델링(값의 범위 내에서 모든 조합을 다 시도한다.)  
③ 가장 성능이 좋은 값을 선정


#### ① 값의 범위를 지정한다.

In [31]:
# dictionary형태로 선언
params = { 'n_neighbors' : range(3,31,2), 'metric' : ['euclidean', 'manhattan']  }
params

{'n_neighbors': range(3, 31, 2), 'metric': ['euclidean', 'manhattan']}

#### ② 모델 선언

In [32]:
# 기본모델
model = KNeighborsClassifier()

# Random Search 설정.
model_gs = GridSearchCV(model, params, cv=5)

#### ③ 모델링

In [33]:
# 학습 : model이 아니라 model_rs
model_gs.fit(x_train_s, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': range(3, 31, 2)})

In [34]:
# 튜닝 결과
model_gs.cv_results_

{'mean_fit_time': array([0.00220532, 0.00314784, 0.00222387, 0.00197401, 0.00222158,
        0.00154262, 0.0017077 , 0.00222635, 0.00259542, 0.00178494,
        0.00219545, 0.00277929, 0.00200057, 0.00219979, 0.00161357,
        0.0019814 , 0.00185056, 0.00164137, 0.00161147, 0.0019371 ,
        0.00156631, 0.00149841, 0.00156832, 0.00118895, 0.00152073,
        0.00237818, 0.00258255, 0.00279961]),
 'std_fit_time': array([0.00039508, 0.00192747, 0.00038646, 0.00013412, 0.00039307,
        0.00056543, 0.00036365, 0.00097148, 0.00081982, 0.00040694,
        0.00076792, 0.00066924, 0.0010922 , 0.0007385 , 0.00046808,
        0.00068034, 0.00041195, 0.00055816, 0.00037578, 0.00136196,
        0.0005249 , 0.00044634, 0.00054259, 0.00038981, 0.00086636,
        0.0013597 , 0.0017492 , 0.00212915]),
 'mean_score_time': array([0.10562644, 0.1190062 , 0.12048712, 0.10403876, 0.10503707,
        0.1052207 , 0.11230321, 0.13342309, 0.11725149, 0.09938478,
        0.10191307, 0.11040101, 0.107515

In [35]:
model_gs.cv_results_['params']

[{'metric': 'euclidean', 'n_neighbors': 3},
 {'metric': 'euclidean', 'n_neighbors': 5},
 {'metric': 'euclidean', 'n_neighbors': 7},
 {'metric': 'euclidean', 'n_neighbors': 9},
 {'metric': 'euclidean', 'n_neighbors': 11},
 {'metric': 'euclidean', 'n_neighbors': 13},
 {'metric': 'euclidean', 'n_neighbors': 15},
 {'metric': 'euclidean', 'n_neighbors': 17},
 {'metric': 'euclidean', 'n_neighbors': 19},
 {'metric': 'euclidean', 'n_neighbors': 21},
 {'metric': 'euclidean', 'n_neighbors': 23},
 {'metric': 'euclidean', 'n_neighbors': 25},
 {'metric': 'euclidean', 'n_neighbors': 27},
 {'metric': 'euclidean', 'n_neighbors': 29},
 {'metric': 'manhattan', 'n_neighbors': 3},
 {'metric': 'manhattan', 'n_neighbors': 5},
 {'metric': 'manhattan', 'n_neighbors': 7},
 {'metric': 'manhattan', 'n_neighbors': 9},
 {'metric': 'manhattan', 'n_neighbors': 11},
 {'metric': 'manhattan', 'n_neighbors': 13},
 {'metric': 'manhattan', 'n_neighbors': 15},
 {'metric': 'manhattan', 'n_neighbors': 17},
 {'metric': 'manha

In [36]:
model_gs.cv_results_['mean_test_score']

array([0.57885714, 0.57942857, 0.57714286, 0.57342857, 0.568     ,
       0.57457143, 0.58085714, 0.58257143, 0.57914286, 0.58057143,
       0.586     , 0.58457143, 0.57971429, 0.58028571, 0.57314286,
       0.59142857, 0.59171429, 0.59142857, 0.59828571, 0.59771429,
       0.59942857, 0.60142857, 0.60171429, 0.61057143, 0.61      ,
       0.61142857, 0.61857143, 0.618     ])

In [37]:
# 최적의 파라미터
model_gs.best_params_

{'metric': 'manhattan', 'n_neighbors': 27}

In [38]:
# 그때의 성능
model_gs.best_score_

0.6185714285714285

In [39]:
# best 모델로 예측 및 평가
pred = model_gs.predict(x_val_s)
print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       0.59      0.72      0.65       738
           1       0.65      0.51      0.57       762

    accuracy                           0.61      1500
   macro avg       0.62      0.61      0.61      1500
weighted avg       0.62      0.61      0.61      1500



### (5) 실습 : Grid Search

* decision tree로 튜닝을 시도해 봅시다.
    * max_depth : 1~10
    * min_samples_leaf : 10 ~ 100

#### ① 값의 범위를 지정한다.

In [65]:
params = {'max_depth' : range(1, 11), 'min_samples_leaf' : range(10, 101, 10)}

#### ② 모델 선언

In [79]:
model = DecisionTreeClassifier()
model_gs = GridSearchCV(model, params, cv = 5, verbose = 3)

#### verbose
    Controls the verbosity: the higher, the more messages.

    - >1 : the computation time for each fold and parameter candidate is
      displayed;
    - >2 : the score is also displayed;
    - >3 : the fold and candidate parameter indexes are also displayed
      together with the starting time of the computation.

#### ③ 모델링

In [80]:
model_gs.fit(x_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 1/5] END ...............max_depth=1, min_samples_leaf=10; total time=   0.0s
[CV 2/5] END ...............max_depth=1, min_samples_leaf=10; total time=   0.0s
[CV 3/5] END ...............max_depth=1, min_samples_leaf=10; total time=   0.0s
[CV 4/5] END ...............max_depth=1, min_samples_leaf=10; total time=   0.0s
[CV 5/5] END ...............max_depth=1, min_samples_leaf=10; total time=   0.0s
[CV 1/5] END ...............max_depth=1, min_samples_leaf=20; total time=   0.0s
[CV 2/5] END ...............max_depth=1, min_samples_leaf=20; total time=   0.0s
[CV 3/5] END ...............max_depth=1, min_samples_leaf=20; total time=   0.0s
[CV 4/5] END ...............max_depth=1, min_samples_leaf=20; total time=   0.0s
[CV 5/5] END ...............max_depth=1, min_samples_leaf=20; total time=   0.0s
[CV 1/5] END ...............max_depth=1, min_samples_leaf=30; total time=   0.0s
[CV 2/5] END ...............max_depth=1, min_s

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(1, 11),
                         'min_samples_leaf': range(10, 101, 10)},
             verbose=3)

In [57]:
model_gs.best_params_

{'max_depth': 7, 'min_samples_leaf': 30}

In [58]:
model_gs.best_score_

0.7005714285714285

In [59]:
model_gs.cv_results_['mean_test_score'] # 시도했을 때 성능 값

array([0.614     , 0.614     , 0.614     , 0.614     , 0.614     ,
       0.614     , 0.614     , 0.614     , 0.614     , 0.614     ,
       0.66142857, 0.66142857, 0.66142857, 0.66142857, 0.66142857,
       0.66142857, 0.66142857, 0.66142857, 0.66142857, 0.66142857,
       0.692     , 0.69314286, 0.69314286, 0.69314286, 0.69314286,
       0.69314286, 0.69314286, 0.69314286, 0.69314286, 0.69514286,
       0.68942857, 0.68857143, 0.68942857, 0.68971429, 0.68942857,
       0.68914286, 0.69285714, 0.69342857, 0.69742857, 0.69314286,
       0.686     , 0.69285714, 0.69514286, 0.69314286, 0.68857143,
       0.688     , 0.69057143, 0.69085714, 0.69771429, 0.69342857,
       0.67857143, 0.68428571, 0.68714286, 0.68657143, 0.69114286,
       0.68742857, 0.69142857, 0.68971429, 0.69571429, 0.69228571,
       0.67314286, 0.688     , 0.70057143, 0.69628571, 0.68971429,
       0.686     , 0.69      , 0.69114286, 0.69542857, 0.69228571,
       0.66571429, 0.688     , 0.694     , 0.694     , 0.69228

In [60]:
print(classification_report(y_val, pred_gs))

              precision    recall  f1-score   support

           0       0.66      0.70      0.68       738
           1       0.69      0.66      0.68       762

    accuracy                           0.68      1500
   macro avg       0.68      0.68      0.68      1500
weighted avg       0.68      0.68      0.68      1500



In [82]:
print(accuracy_score(y_val, pred_gs))

0.6786666666666666


In [ ]:
# 데이터가 다르기 때문에 model_gs.best_score_와 accuracy_score(y_val, pred_gs)가 다를 수 있음